In [ ]:
#from importlib.resources import path
import os
import torch
import clip
import numpy as np
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
from tqdm import tqdm
import pandas as pd
import time
from PIL import Image
from PIL import ImageFile  # 大きな画像もロード
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
model = model.eval()

In [ ]:
images_dir = "./imagenet_fall2011_oneImagePerCat_21-2k_20230309/"
list_images = os.listdir(images_dir)
len(list_images)

In [ ]:
image_names_list = []
image_feats_list = []

for img in list_images:
    img_dir = images_dir + img
    image = preprocess(Image.open(img_dir)).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)

    
    image_features = image_features.to('cpu').detach().numpy().copy()
    
    image_feats_list.append(image_features)
    image_names_list.append(img)

In [ ]:
image_names_np = np.array(image_names_list)
image_feats_np = np.array(image_feats_list)

In [ ]:
image_feats_np = np.squeeze(image_feats_np)

In [ ]:
name_feat_dict = {}
for img_name  in image_names_list:
    for feat_val in image_feats_list:
        name_feat_dict[img_name] = feat_val

In [ ]:
a_file = open("./normal_semantic_features/stim_ID_clip_imagenet19k_map_20230318.pkl", "wb")
pickle.dump(name_feat_dict, a_file)
a_file.close()